In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [10]:
# create s3 bucket
# 1. The bucket name can be between 3 and 63 characters long,
# and can contain only lower-case characters, numbers,
# periods, and dashes.
# 2. Bucket namespace is shared among users so it needs to be unique
bucket_name = 'bank-application-jb'
my_region = boto3.session.Session().region_name
print(my_region)

us-east-2


In [11]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-2':
        s3.create_bucket(Bucket=bucket_name,
            CreateBucketConfiguration={"LocationConstraint": "us-east-2"})
        print(f'S3 buckt {bucket_name} created successfully!')
except Exception as e:
    print('S3 error: ', e)

S3 buckt bank-application-jb created successfully!


In [12]:
# set output path where the trained model will be stored
prefix = 'xgboost-model'
output_path = f's3://{bucket_name}/{prefix}/output'
print(output_path)

s3://bank-application-jb/xgboost-model/output


In [17]:
# import modules
import pandas as pd
import numpy as np
import urllib
import os

In [14]:
# Download data
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [37]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=19), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [21]:
# rearrange train/test data to Sagemaker's requirement
train_data = pd.concat([train_data['y_yes'], train_data.drop(columns=['y_no', 'y_yes'])], 
                                                axis=1)
test_data = pd.concat([test_data['y_yes'], test_data.drop(columns=['y_no', 'y_yes'])], 
                                                axis=1)
train_data.to_csv('train_data.csv', index=False, header=False)
test_data.to_csv('test_data.csv', index=False, header=False)

In [22]:
# store files to s3 bucket
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train_data.csv')).upload_file('train_data.csv')
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test_data.csv')).upload_file('test_data.csv')

In [24]:
# reference data from s3 bucket
s3_input_train = sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket_name}/{prefix}/train', content_type='csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket_name}/{prefix}/test', content_type='csv')

Use Sagemaker Builtin Algorithm

In [26]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.7-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [31]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50,
        "early_stopping_rounds": 7
        }

In [32]:

# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600)

In [33]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2025-07-30-19-07-53-438


2025-07-30 19:07:57 Starting - Starting the training job...
2025-07-30 19:08:28 Downloading - Downloading input data...
2025-07-30 19:08:44 Downloading - Downloading the training image...
2025-07-30 19:09:19 Training - Training image download completed. Training in progress...[2025-07-30 19:09:40.150 ip-10-0-129-137.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node trainin

Deploy Model

In [34]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')    

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-07-30-19-13-11-839
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2025-07-30-19-13-11-839
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2025-07-30-19-13-11-839


------!

Prediction

In [38]:
from sagemaker.serializers import CSVSerializer
predict_data = test_data.drop(columns=['y_no', 'y_yes']).values # reusing test data for prediction
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(predict_data).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [39]:
predictions_array

array([0.19996458, 0.19996458, 0.23559932, ..., 0.23559932, 0.19996458,
       0.19996458])

In [40]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.8%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10829)    33% (131)
Purchase        9% (1135)     67% (262) 



Endpoint deletion

In [41]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2025-07-30-19-13-11-839


[{'ResponseMetadata': {'RequestId': 'DVWMDFA4JB0JYQHX',
   'HostId': 'MiThPWlzIYQDiBqcyfmL0Rc67YTWPhb+FJ5e6EakLGhqEXmrioQTj1c6H4bATfA4l9N8F8Z3g+Di2IsWUqmQfA==',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'MiThPWlzIYQDiBqcyfmL0Rc67YTWPhb+FJ5e6EakLGhqEXmrioQTj1c6H4bATfA4l9N8F8Z3g+Di2IsWUqmQfA==',
    'x-amz-request-id': 'DVWMDFA4JB0JYQHX',
    'date': 'Wed, 30 Jul 2025 19:26:44 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-model/output/sagemaker-xgboost-2025-07-30-19-07-53-438/debug-output/index/000000000/000000000000_worker_0.json'},
   {'Key': 'xgboost-model/output/sagemaker-xgboost-2025-07-30-19-07-53-438/debug-output/index/000000000/000000000010_worker_0.json'},
   {'Key': 'xgboost-model/output/sagemaker-xgboost-2025-07-30-19-07-53-438/profiler-output/framework/training_job_end.ts'},
   {'Key': 'xgboost-model/output/sagemaker-